In [10]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
# 1. 데이터 로드 및 전처리
file_path = 'eth_4hour_all.csv'
df = pd.read_csv(file_path)

def load_and_standardize(path):
    if not os.path.exists(path):
        print(f"❌ File not found: {path}")
        return pd.DataFrame()

    df = pd.read_csv(path)
    df.columns = df.columns.str.strip().str.lower()
    
    rename_map = {}
    for col in df.columns:
        if 'eth_' in col: rename_map[col] = col.replace('eth_', '')
    if rename_map: df.rename(columns=rename_map, inplace=True)
        
    time_col = [c for c in df.columns if 'time' in c or 'date' in c][0]
    df[time_col] = pd.to_datetime(df[time_col])
    df.set_index(time_col, inplace=True)
    
    # 중복 제거 및 정렬
    df = df[~df.index.duplicated(keep='first')].sort_index()
    return df

df=load_and_standardize(file_path)
# 컬럼 이름 소문자 변환 및 정렬 (필요시 수정)
df.columns = [c.lower() for c in df.columns] 
df = df.sort_values('date').reset_index(drop=True)

# 2. ATR 계산 함수 (이미 사용 중이시겠지만 검증용으로 독립 정의)
def calculate_atr(data, period=14):
    high_low = data['high'] - data['low']
    high_close = np.abs(data['high'] - data['close'].shift())
    low_close = np.abs(data['low'] - data['close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = ranges.max(axis=1)
    return true_range.rolling(window=period).mean()

df['ATR'] = calculate_atr(df)

# 파라미터 정의
TIME_HORIZONS = [4, 8, 12, 24, 48, 72, 96, 168]
RATIOS = [
    (0.5, 0.5, 'QuickScalp'),
    (0.8, 0.8, 'Balanced'),
    (1.0, 0.5, 'Sniper'),
    (1.5, 1.0, 'Standard'),
    (2.0, 1.0, 'Classic'),
    (2.5, 1.5, 'Growth'),
    (3.0, 1.5, 'Trend'),
    (6.0, 3.0, 'BigTrend'),
    (10.0, 10.0, 'Jackpot'),
]

# 3. 타당성 분석 (Feasibility Loop)
feasibility_results = []

print("Running Feasibility Check (Volatility Analysis)...")

for horizon in tqdm(TIME_HORIZONS, desc="Checking Horizons"):
    # 미래 horizon 기간 동안의 최고가(High)와 최저가(Low)를 미리 계산 (Look-ahead)
    # pd.api.indexers.FixedForwardWindowIndexer를 사용하여 미래 데이터 참조
    indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=horizon)
    
    # 현재 시점 이후 horizon 기간 동안의 최대 상승폭 (MFE: Max Favorable Excursion)
    future_high = df['high'].rolling(window=indexer).max()
    max_upside_atr = (future_high - df['close']) / df['ATR']
    
    # 현재 시점 이후 horizon 기간 동안의 최대 하락폭 (MAE: Max Adverse Excursion)
    future_low = df['low'].rolling(window=indexer).min()
    max_downside_atr = (df['close'] - future_low) / df['ATR']
    
    # 각 Ratio 조합에 대해 도달 가능성 계산
    for tp, sl, strategy_name in RATIOS:
        # TP 도달 확률: 기간 내 최대 상승폭이 TP 배수보다 클 확률
        tp_prob = (max_upside_atr >= tp).mean()
        
        # SL 도달 확률: 기간 내 최대 하락폭이 SL 배수보다 클 확률
        sl_prob = (max_downside_atr >= sl).mean()
        
        # 유효 샘플 수 (TP나 SL 중 하나라도 건드리는 비율 -> 레이블링 가능한 데이터 비율)
        active_ratio = ((max_upside_atr >= tp) | (max_downside_atr >= sl)).mean()
        
        feasibility_results.append({
            'Horizon(Hours)': horizon,
            'Strategy': strategy_name,
            'TP_Ratio': tp,
            'TP_Prob(%)': round(tp_prob * 100, 2),
            'SL_Ratio': sl,
            'SL_Prob(%)': round(sl_prob * 100, 2),
            'Active_Data_Rate(%)': round(active_ratio * 100, 2),
            'Status': 'Review' # 아래 로직에서 판별
        })

# 4. 결과 분석 및 필터링
res_df = pd.DataFrame(feasibility_results)

def judge_validity(row):
    # 기준: 데이터의 최소 5% 이상은 트리거되어야 학습 가능 (레이블 불균형 방지)
    if row['TP_Prob(%)'] < 1.0: # 1% 미만이면 사실상 불가능한 목표 (Outlier)
        return "Impossible (Discard)"
    elif row['TP_Prob(%)'] < 5.0:
        return "Very Hard (High Risk)"
    elif row['TP_Prob(%)'] > 80.0:
        return "Too Easy (Noise)"
    else:
        return "Optimal"

res_df['Status'] = res_df.apply(judge_validity, axis=1)

# 5. 결과 출력 (Pivot Table 형태로 보기 쉽게)
print("\n[Analysis Report: TP Probability per Horizon]")
pivot_table = res_df.pivot(index='Strategy', columns='Horizon(Hours)', values='TP_Prob(%)')
print(pivot_table)

print("\n[Recommendation]")
print("삭제 권장 (TP 확률 1% 미만):")
print(res_df[res_df['Status'] == "Impossible (Discard)"][['Horizon(Hours)', 'Strategy', 'TP_Prob(%)']])


Running Feasibility Check (Volatility Analysis)...


Checking Horizons: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 125.42it/s]


[Analysis Report: TP Probability per Horizon]
Horizon(Hours)    4      8      12     24     48     72     96     168
Strategy                                                              
Balanced        50.13  64.89  71.86  80.30  85.68  87.66  89.22  91.18
BigTrend         0.40   2.26   4.98  13.10  25.45  33.93  40.02  52.28
Classic         11.50  24.42  33.12  49.11  62.16  68.37  72.44  78.00
Growth           6.85  17.25  25.23  40.49  54.82  62.03  66.83  73.48
Jackpot          0.03   0.40   1.08   4.78  12.94  19.46  25.16  38.45
QuickScalp      73.19  82.10  85.90  90.28  93.04  93.91  94.48  95.15
Sniper          38.21  54.83  62.99  73.95  80.94  83.87  85.83  88.67
Standard        20.32  36.11  45.34  59.80  70.56  75.24  78.62  82.93
Trend            4.30  12.46  19.48  33.71  48.83  56.62  62.12  69.47

[Recommendation]
삭제 권장 (TP 확률 1% 미만):
    Horizon(Hours)  Strategy  TP_Prob(%)
7                4  BigTrend        0.40
8                4   Jackpot        0.03
17        